In [1]:
# %load most-popular-superhero.py
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("PopularHero")
sc = SparkContext(conf = conf)

In [3]:
# Split a line on whitespace and return a tuple of the character ID 
# and the number of other characters
def countCoOccurences(line):
    elements = line.split()
    # Don't want to include the hero ID (len(elements)-1)
    return (int(elements[0]), len(elements) - 1)

# Not sure if I understand how split is being used here
def parseNames(line):
    fields = line.split('\"')
    return (int(fields[0]), fields[1].encode("utf8"))

In [17]:
names = sc.textFile("file:///home/dmadhok/spark_course/Marvel-Names.txt")
namesRdd = names.map(parseNames)

In [18]:
namesRdd.take(5)

[(1, b'24-HOUR MAN/EMMANUEL'),
 (2, b'3-D MAN/CHARLES CHAN'),
 (3, b'4-D MAN/MERCURIO'),
 (4, b'8-BALL/'),
 (5, b'A')]

In [19]:
lines = sc.textFile("file:///home/dmadhok/spark_course/Marvel-Graph.txt")
lines.take(5)

['5988 748 1722 3752 4655 5743 1872 3413 5527 6368 6085 4319 4728 1636 2397 3364 4001 1614 1819 1585 732 2660 3952 2507 3891 2070 2239 2602 612 1352 5447 4548 1596 5488 1605 5517 11 479 2554 2043 17 865 4292 6312 473 534 1479 6375 4456 ',
 '5989 4080 4264 4446 3779 2430 2297 6169 3530 3272 4282 6432 2548 4140 185 105 3878 2429 1334 4595 2767 3956 3877 4776 4946 3407 128 269 5775 5121 481 5516 4758 4053 1044 1602 3889 1535 6038 533 3986 ',
 '5982 217 595 1194 3308 2940 1815 794 1503 5197 859 5096 6039 2664 651 2244 528 284 1449 1097 1172 1092 108 3405 5204 387 4607 4545 3705 4930 1805 4712 4404 247 4754 4427 1845 536 5795 5978 533 3984 6056 ',
 '5983 1165 3836 4361 1282 716 4289 4646 6300 5084 2397 4454 1913 5861 5485 ',
 '5980 2731 3712 1587 6084 2472 2546 6313 875 859 323 2664 1469 522 2506 2919 2423 3624 5736 5046 1787 5776 3245 3840 2399 ']

In [20]:
# Remember to try shift tab :)

# (Character ID, characters that character ID coocurs with)
pairings = lines.map(countCoOccurences)
pairings.take(5)

[(5988, 48), (5989, 40), (5982, 42), (5983, 14), (5980, 24)]

In [13]:
# Same character ID can appear on multiple lines- so we need to 
# reduce by key
totalFriendsByCharacter = pairings.reduceByKey(lambda x, y : x + y)
totalFriendsByCharacter.take(5)

[(5988, 48), (5989, 40), (5982, 42), (5983, 14), (5980, 24)]

In [14]:
# Flip the above representation
flipped = totalFriendsByCharacter.map(lambda xy : (xy[1], xy[0]))
flipped.take(5)

[(48, 5988), (40, 5989), (42, 5982), (14, 5983), (24, 5980)]

In [15]:
# 859 is the character ID with the max # of coocurrences
mostPopular = flipped.max()
mostPopular

(1933, 859)

In [24]:
# Get the name of character ID 859 using the lookup function
mostPopularName = namesRdd.lookup(mostPopular[1])[0]
mostPopularName

b'CAPTAIN AMERICA'

In [25]:
print(str(mostPopularName) + " is the most popular superhero, with " + \
    str(mostPopular[0]) + " co-appearances.")

b'CAPTAIN AMERICA' is the most popular superhero, with 1933 co-appearances.
